In [6]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [7]:
import numpy as np
import tensorflow
from tensorflow import keras
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Attention

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 


# Read the Twitter dataset from 'twitter_data.txt' file
data_file = 'schrute.txt'
with open(data_file, 'r', encoding='utf-8') as f:
    tweet_lines = f.readlines()

# Split the dataset into input and target texts
input_texts = []
target_texts = []
for tweet in tweet_lines:
    input_texts.append(tweet)
    target_texts.append('\t' + tweet + '\n')

# Define the input and target characters based on your dataset
input_characters = set()
target_characters = set()

for input_text, target_text in zip(input_texts, target_texts):
    input_characters.update(input_text)
    target_characters.update(target_text)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# Rest of the code remains the same as before (tokenization, vectorization, model definition, training)

# Creating dictionaries to map characters to integers
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

max_encoder_seq_length = max([len(text) for text in input_texts])
max_decoder_seq_length = max([len(text) for text in target_texts])

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# Creating empty arrays to store encoder and decoder data
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# Vectorizing the input and target texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

# Define the model architecture with attention
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

attention = Attention()
attention_outputs = attention([decoder_outputs, encoder_outputs])

decoder_concat_input = keras.layers.concatenate([decoder_outputs, attention_outputs])
decoder_dense = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=32,  # Try reducing the batch size
    epochs=50,
    validation_split=0.2
)

# Save the trained model
model.save('the_office_chatbot_model.h5')


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [12]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
